# Subtractive Synthesis using Linear Predictive Coding 


We have seen that the overall spectral shape plays an important role in how we perceive sounds. We can use sinusoids and additive synthesis to model spectral shape but we also saw how we can use resonant-filters centered at the spectral modes of a system to generate realistic sounds when we looked at modal synthesis. 

The center frequencies and resonances of these filters were determined empirically through looking at spectral plots. Linear Predictive Coding is a technique. 

In [ ]:
import IPython.display as ipd 
import sys
import pyaudio
import numpy as np
import scipy
from bokeh.io import output_notebook
from bokeh.plotting import figure, output_file, show

output_notebook()

# plot_library = 'matplotlib'
plot_library = 'matplotlib_xkcd'
# plot_library = 'bokeh'


if (plot_library=='bokeh'):
    import bokeh 
    from bokeh.io import output_notebook
    from bokeh.plotting import figure, output_file, show
    output_notebook()

    def plot(data_list): 
        p = figure(plot_height=300, plot_width=600, title='Synthesizers')
        for data in data_list: 
            p.line(np.arange(0,len(data)), data)
        show(p)
        
if (plot_library=='matplotlib'): 
    %matplotlib notebook 
    import matplotlib.pyplot as plt
    def plot(data_list,label_list=[],xlabel='', ylabel='', title=''):
        fig, ax = plt.subplots(figsize=(8,4))
        for (data,label) in zip(data_list, label_list): 
            plt.title('Synth-CS: '+title)
            plt.xlabel(xlabel)
            plt.ylabel(ylabel)
            plt.plot(np.arange(0, len(data)), data, label=label)
        if (label_list):
            ax.legend()
        plt.ion()
        plt.show()
        
if (plot_library=='matplotlib_xkcd'): 
    %matplotlib notebook 
    import matplotlib.pyplot as plt

    def plot(data_list,label_list=[],xlabel='', ylabel='', title=''):
        fig, ax = plt.subplots(figsize=(5,3))  
        plt.xkcd()
        if not(label_list):
            for (i,d) in enumerate(data_list): 
                label_list.append(str(i))
        for (data,label) in zip(data_list, label_list): 
            plt.title('Synth-CS: '+title)
            plt.xlabel(xlabel)
            plt.ylabel(ylabel)
            plt.plot(np.arange(0, len(data)), data, label=label)  
            ax.legend()
        plt.ion()
        plt.show()

In [ ]:
import librosa
import IPython.display as ipd

In [ ]:
y, sr = librosa.load("lpc_example.wav")
ipd.Audio(y, rate=sr)

In [ ]:
def overlap_add(input, win_size, hop_size): 
    # precompute the hamming window
    window = scipy.signal.hann(win_size)
    output= np.zeros(len(input)+win_size)   

   # plot([window])
    for i in range(0,len(input),hop_size):
        # slice the audio into overlapping frames 
        frame = input[i:i+win_size]
       # if (i < 1024): 
       #     plot([frame])
       #     plot([frame * window])
        if len(frame)==win_size: 
            output[i:i+win_size] += (frame * window)  
            
            
    return 0.5 * output    

output = overlap_add(y, 1024, 512)
ipd.Audio(output, rate=sr, normalize=False)

In [ ]:
# This function is copied directly from https://github.com/cournape/talkbox/blob/master/scikits/talkbox/linpred/py_lpc.py
# Copyright (c) 2008 Cournapeau David
# (MIT licensed)
def levinson_1d(r, order):
    """Levinson-Durbin recursion, to efficiently solve symmetric linear systems
    with toeplitz structure.

    Parameters
    ---------
    r : array-like
        input array to invert (since the matrix is symmetric Toeplitz, the
        corresponding pxp matrix is defined by p items only). Generally the
        autocorrelation of the signal for linear prediction coefficients
        estimation. The first item must be a non zero real.

    Notes
    ----
    This implementation is in python, hence unsuitable for any serious
    computation. Use it as educational and reference purpose only.

    Levinson is a well-known algorithm to solve the Hermitian toeplitz
    equation:

                       _          _
        -R[1] = R[0]   R[1]   ... R[p-1]    a[1]
         :      :      :          :      *  :
         :      :      :          _      *  :
        -R[p] = R[p-1] R[p-2] ... R[0]      a[p]
                       _
    with respect to a (  is the complex conjugate). Using the special symmetry
    in the matrix, the inversion can be done in O(p^2) instead of O(p^3).
    """
    r = np.atleast_1d(r)
    if r.ndim > 1:
        raise ValueError("Only rank 1 are supported for now.")

    n = r.size
    if n < 1:
        raise ValueError("Cannot operate on empty array !")
    elif order > n - 1:
        raise ValueError("Order should be <= size-1")

    if not np.isreal(r[0]):
        raise ValueError("First item of input must be real.")
    elif not np.isfinite(1/r[0]):
        raise ValueError("First item should be != 0")

    # Estimated coefficients
    a = np.empty(order+1, r.dtype)
    # temporary array
    t = np.empty(order+1, r.dtype)
    # Reflection coefficients
    k = np.empty(order, r.dtype)

    a[0] = 1.
    e = r[0]

    for i in range(1, order+1):
        acc = r[i]
        for j in range(1, i):
            acc += a[j] * r[i-j]
        k[i-1] = -acc / e
        a[i] = k[i-1]

        for j in range(order):
            t[j] = a[j]

        for j in range(1, i):
            a[j] += k[i-1] * np.conj(t[i-j])

        e *= 1 - k[i-1] * np.conj(k[i-1])

    return a, e, k



def lpc(wave, order):
    """Compute LPC of the waveform. 
    a: the LPC coefficients
    e: the total error
    k: the reflection coefficients
    
    Typically only a is required.
    """    
    # only use right half of autocorrelation, normalised by total length
    autocorr = scipy.signal.correlate(wave, wave)[len(wave)-1:]/len(wave)
    a, e, k  = levinson_1d(autocorr, order)
    return a,e,k

In [ ]:
def lpc_process(input, win_size, hop_size, order,excitation): 
    # precompute the hamming window
    window = scipy.signal.hann(win_size)
    output= np.zeros(len(input)+win_size)   

    # plot([window])
    for i in range(0,len(input),hop_size):
        # slice the audio into overlapping frames 
        frame = input[i:i+win_size]
        excitation_frame = excitation[i:i+win_size]
       
        if len(frame)==win_size: 
            a,error,reflection = lpc(frame, order)    
            lpc_residual = scipy.signal.lfilter(a, 1., frame) 
            # excitation_frame = lpc_residual
            lpc_output = scipy.signal.lfilter([1.], a, excitation_frame)             
            voc_amp = 1e-5+np.sqrt(np.mean(lpc_output**2))
            wave_amp = 1e-5+np.sqrt(np.mean(frame**2))
            lpc_output = lpc_output * (wave_amp/voc_amp)
            output[i:i+win_size] += (lpc_output * window)  
            
            
    return output    


def modfm_buzz(samples, f, sr, k):
    """Generate a pulse train using modfm:
        y(t) = cos(x(t)) * exp(cos(x(t))*k - k)
        
        samples: number of samples to generate
        f: base frequency (Hz)
        sr: sample rate (Hz)
        k: modulation depth; higher has more harmonics but increases risk of aliasing
        (e.g. k=1000 for f=50, k=100 for f=200, k=2 for f=4000)        
    
    """
    t = np.arange(samples)
    phase = (f*2*np.pi * (t/float(sr)))
    # simple pulse oscillator (ModFM)
    buzz = np.cos(phase) * np.exp(np.cos(phase)*k-k)    
    return buzz


# ModFM pulse train, with exp. decreasing modulation depth (lowpass filter effect)
buzz = modfm_buzz(len(y), f=100*np.floor(np.linspace(1,6,len(y)))**0.25,
                     sr=sr, k=10**np.linspace(4,2,len(y)))
ipd.Audio(buzz, rate=sr)




In [ ]:
order = 48

  

In [ ]:
output = lpc_process(y, 1024, 512, order, buzz)
ipd.Audio(output, rate=sr)

In [ ]:
order = 48
noise = np.random.uniform(0,0.5,len(y))
output = lpc_process(y, 1024, 512, order, noise)
ipd.Audio(output, rate=sr)

In [ ]:
sawtooth = scipy.signal.sawtooth(2*np.pi*200.0*np.arange(len(y))/(sr))
output = lpc_process(y, 1024, 512, order, sawtooth)
ipd.Audio(output, rate=sr)

In [ ]:
ipd.Audio(output, rate=44100.0)

In [ ]:
impulse = scipy.signal.unit_impulse(len(y))
output = lpc_process(y, 1024, 512, order, impulse)
ipd.Audio(output, rate=sr)